In [7]:
from serpapi import GoogleSearch
from langchain_community.document_loaders import ScrapingAntLoader
import dotenv
import os
dotenv.load_dotenv()
serp_api = os.getenv("SERP_API_KEY")
ant_scrap = os.getenv("ANT_SCRAPY_API_KEY")

In [21]:
def flight_details(
    from_location_ID: str, 
    to_location_ID: str, 
    departure_date: str, ):
    """
    Fetches flight details using the Google Flights API.

    Args:
    from_location_ID (str): The departure location ID (e.g., airport code like 'DEL').
    to_location_ID (str): The destination location ID (e.g., airport code like 'AUS').
    departure_date (str): The departure date in 'YYYY-MM-DD' format.
    return_date (str): The return date in 'YYYY-MM-DD' format.

    Returns:
    tuple: A tuple containing two lists:
        - best_flights: A list of the best flight options.
        - other_flights: A list of other available flight options.
    """
    params = {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "departure_id": from_location_ID,
    "arrival_id": to_location_ID,
    "outbound_date": departure_date,
    "currency": "INR",
    "type" : 2,
    "api_key": serp_api,  
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [22]:
def hotel_details(
    stay_city_and_type: str,
    check_in_date: str,
    check_out_date: str,
    num_adults: int,
    min_price: int,
    max_price: int,
    num_children: int = 0,
    children_ages: list = [0],
    page: int = 3,
):
    """
    Fetches hotel details using the Google Hotels API.

    Args:
        stay_city_and_type (str): The city and type of stay (e.g., 'Hotels in Mumbai').
        check_in_date (str): The check-in date in 'YYYY-MM-DD' format.
        check_out_date (str): The check-out date in 'YYYY-MM-DD' format.
        num_adults (int): The number of adults staying.
        min_price (int): The minimum price for the stay.
        max_price (int): The maximum price for the stay.
        num_children (int, optional): The number of children staying. Defaults to 0.
        children_ages (list, optional): A list of ages of the children. Defaults to [0].
        page (int, optional): The page number for paginated results. Defaults to 3.

    Returns:
        list: A list of dictionaries containing details about the hotels, such as name, price, rating, etc.
    """
    all_properties = []

    for _ in range(3):  
        params = {
            "engine": "google_hotels",
            "hl": "en",
            "gl": "us",
            "q": stay_city_and_type,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": num_adults,
            "children": num_children,
            "children_ages": children_ages,
            "min_price": min_price,
            "max_price": max_price,
            "currency": "INR",
            "api_key": serp_api,
            "start": page * 10,
        }

        search = GoogleSearch(params)
        results = search.get_dict()
        properties = results.get("properties", [])

        if not properties:
            break  

        all_properties.extend(properties)

    return all_properties


In [23]:
def local_activities(activity_query: str, destination_city: str, num_pages: int = 3):
    """
    Fetches local activity details for a given query and destination city using the Google Local API.

    Args:
        activity_query (str): The type of activity to search for (e.g., 'bar', 'restaurant').
        destination_city (str): The city where the activity is located (e.g., 'Goa').
        num_pages (int, optional): The number of pages of results to fetch. Defaults to 3.

    Returns:
        list: A list of dictionaries containing details about the local activities, such as name, address, rating, etc.
    """
    all_places = []
    for page in range(num_pages):
        params = {
            "api_key": serp_api,  
            "engine": "google_local",
            "google_domain": "google.com",
            "q": f"{activity_query} in {destination_city}",
            "hl": "en",
            "gl": "us",
            "start": num_pages * 20
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        places = results.get("local_results", [])
        if not places:
            break
        all_places.extend(places)

    return all_places

In [24]:
def get_train_details(departure_code: str, departure_name: str,
                           destination_code: str, destination_name: str,
                           journey_date: str) -> str:
    """
    Load train search HTML page from Railyatri using ScrapingAntLoader.
    
    Args:
        departure_code (str): Station code of departure (e.g., 'JBP')
        departure_name (str): Station name of departure (e.g., 'JABALPUR')
        destination_code (str): Station code of destination (e.g., 'BPL')
        destination_name (str): Station name of destination (e.g., 'BHOPAL JN')
        journey_date (str): Date in format '7-4-2025'
        api_key (str): ScrapingAnt API key

    Returns:
        str: HTML page content
    """
    base_url = (
        "https://www.railyatri.in/trains-between-stations-v2"
        f"?from_code={departure_code}"
        f"&from_name={departure_name.upper().replace(' ', '%20')}"
        f"&to_code={destination_code}"
        f"&to_name={destination_name.upper().replace(' ', '%20')}"
        f"&journey_date={journey_date}"
        f"&user_id=-1743734480&user_token=&device_type_id=6"
        f"&src=ttb_landing&from_sta_code={departure_code}&to_sta_code={destination_code}"
    )

    loader = ScrapingAntLoader(
        [base_url],
        api_key=ant_scrap,
        continue_on_failure=True
    )

    docs = loader.load()
    return docs[0].page_content if docs else ""


In [25]:
def get_tourist_place(place_type: str, destination_city: str, num_pages: int = 3):
    """
    Fetches local activity details for a given query and destination city using the Google Local API.

    Args:
        activity_query (str): The type of activity to search for (e.g., 'bar', 'restaurant').
        destination_city (str): The city where the activity is located (e.g., 'Goa').
        num_pages (int, optional): The number of pages of results to fetch. Defaults to 3.

    Returns:
        list: A list of dictionaries containing details about the tourist places with location.
    """
    all_places = []
    for page in range(num_pages):
        params = {
            "api_key": serp_api,  
            "engine": "google_local",
            "google_domain": "google.com",
            "q": f"{place_type} in {destination_city}",
            "hl": "en",
            "gl": "us",
            "start": num_pages * 20
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        places = results.get("local_results", [])
        if not places:
            break
        all_places.extend(places)

    return all_places

In [28]:
result = flight_details("DEL", "BHO", "2025-10-01")


# stay_city_and_type = "Hotels in Mumbai"
# check_in_date = "2025-04-10"
# check_out_date = "2025-04-12"
# num_children = 0
# num_adults = 2
# min_price = 1000
# max_price = 10000

# results = hotel_details(
#     stay_city_and_type,
#     check_in_date,
#     check_out_date,
#     num_children,
#     num_adults,
#     min_price,
#     max_price
# )

In [29]:
result

{'search_metadata': {'id': '67efc762dcff0b09772cf254',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/7b8ce08f8cd31b46/67efc762dcff0b09772cf254.json',
  'created_at': '2025-04-04 11:49:54 UTC',
  'processed_at': '2025-04-04 11:49:54 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=INR&tfs=CBwQAhoeEgoyMDI1LTEwLTAxagcIARIDREVMcgcIARIDQkhPQgEBSAFwAZgBAg&tfu=EgIIAQ',
  'raw_html_file': 'https://serpapi.com/searches/7b8ce08f8cd31b46/67efc762dcff0b09772cf254.html',
  'prettify_html_file': 'https://serpapi.com/searches/7b8ce08f8cd31b46/67efc762dcff0b09772cf254.prettify',
  'total_time_taken': 5.39},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'type': '2',
  'departure_id': 'DEL',
  'arrival_id': 'BHO',
  'outbound_date': '2025-10-01',
  'currency': 'INR'},
 'best_flights': [{'flights': [{'departure_airport': {'name': 'Indira Gandhi International Airport',
      'id': 'DEL',
      'time': '2025-1